In [ ]:
#Yelp 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn import svm
import scipy.sparse as sp
import re
from sklearn.feature_extraction.text import TfidfVectorizer



In [2]:
#load reviews
train = pd.read_csv('yelp_data/yelp_2013_train.csv')
valid = pd.read_csv('yelp_data/yelp_2013_dev.csv')
test = pd.read_csv('yelp_data/yelp_2013_test.csv')


#balance classes
#g = train.groupby('rating')
#train.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))

X_train = train['text'] 
y_train = train['rating']

X_valid = valid['text']
y_valid = valid['rating']
X_test = test['text']
y_test = test['rating']

X_full = X_train + X_valid


In [3]:
max_vocab = 50000
bow_transformer = CountVectorizer(max_features=max_vocab, ngram_range=(1,2), 
                                  binary=True ).fit(X_full.values.astype('U'))

In [16]:
#print(bow_transformer.vocabulary_)

In [4]:
X_train = bow_transformer.transform(X_train.values.astype('U'))
X_valid = bow_transformer.transform(X_valid.values.astype('U'))
X_test = bow_transformer.transform(X_test.values.astype('U'))

In [45]:
max_vocab = 50000
tfidf_transformer = TfidfVectorizer(max_features=max_vocab, ngram_range=(1,2) ).fit(X_full.values.astype('U'))
X_train = tfidf_transformer.transform(X_train.values.astype('U'))
X_valid = tfidf_transformer.transform(X_valid.values.astype('U'))
X_test = tfidf_transformer.transform(X_test.values.astype('U'))

In [60]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)
#X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=101)


In [7]:
from sklearn.naive_bayes import MultinomialNB

X_full = sp.vstack((X_train , X_valid))
y_full = np.append(y_train, y_valid)
nb = MultinomialNB(fit_prior=True)
nb.fit(X_full, y_full)
y_hat = nb.predict(X_test)

print(accuracy_score(y_test, y_hat))

0.5825573065902578


In [15]:
#SVM
def linSVM(parameters, X, T, fold) : 
    n_folds = 3
    svc = LinearSVC(multi_class='ovr', dual=False)
    svc_cv = GridSearchCV(svc, parameters, cv=fold, scoring="accuracy", refit=False)
    svc_cv.fit(X, T)
    scores = svc_cv.cv_results_['mean_test_score']
    params = svc_cv.cv_results_['params']
    print('scores:',scores)
   #print('params:',params)
    return svc_cv.best_params_

def RF(parameters, X, T, fold) : 
    n_folds = 1
    clf = RandomForestClassifier()
    clf_cv = GridSearchCV(clf, parameters, cv=fold, scoring="accuracy", refit=False)
    clf_cv.fit(X, T)
    scores = clf_cv.cv_results_['mean_test_score']
    params = clf_cv.cv_results_['params']
    print('scores:',scores)
    print('params:',params)
    return clf_cv.best_params_

def MNB(parameters, X, T, fold) : 
    n_folds = 5
    mnb = MultinomialNB(fit_prior=True)
    mnb_cv = GridSearchCV(mnb, parameters, cv=fold, scoring="accuracy", refit=False)
    mnb_cv.fit(X, T)
    scores = mnb_cv.cv_results_['mean_test_score']
    params = mnb_cv.cv_results_['params']
    print('scores:',scores)
   #print('params:',params)
    return mnb_cv.best_params_

#placeX = X_valid
#placey = y_valid
#X_valid = X_test
#y_valid = y_test
#X_test = placeX
#y_test = placey

#Predefined Split for CV
my_test_fold = []
for i in range(X_train.shape[0]):
    my_test_fold.append(-1)
for i in range(X_valid.shape[0]):
    my_test_fold.append(0)

fold = PredefinedSplit(test_fold=my_test_fold)
CV_split_T = np.append(y_train, y_valid)

CV_split_X = sp.vstack((X_train , X_valid))  # NOT np.vstack


print(CV_split_T.shape)
print(CV_split_X.shape)


(301514,)
(301514, 50000)


In [52]:
#MNB
alphas = [ 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
9]

param_grid = {'alpha': alphas}

bestParams = MNB(param_grid, CV_split_X, CV_split_T, fold)

nb = MultinomialNB(alpha=bestParams['alpha'], fit_prior=True)
nb.fit(X_train, y_train)
y_hat = nb.predict(X_test)

print(accuracy_score(y_test, y_hat))

scores: [0.58141548 0.58138563 0.58153488 0.58147518 0.58153488 0.58138563
 0.58147518 0.58165428]
0.5863777459407832


In [43]:
#LINSVM
Cs = [ 0.0001, 0.001, 0.01, 0.1, 1]
Cs = [ .002, .003, .004, .005, .006, .007, .008, .009]
Cs = [ .0005, .0006, .0007, .0008, .0009, .0011, .0012, .0013]
#Cs = [.0045, .0047, .0049, .0051, .0053, .0055, ]
#Cs = [.0054, .0056, .0057, .0058, .0059 ]
#Cs = [.0071, .0072, .0073, .0074, .0075, .0076, .0077, .0078, .0079, .0080, .0081, .0081,.0083]
#Cs = [.00750, .00751, .00752, .00753, .00754,.00755,.00756]

param_grid = {'C': Cs}

bestParams = linSVM(param_grid, CV_split_X, CV_split_T, fold)

svc = LinearSVC(C = bestParams['C', multi_class='ovr')
svc.fit(X_train, y_train)

y_hat = svc.predict(X_test)
print(accuracy_score(y_test, y_hat))

scores: [0.59508671 0.59744485 0.59929554 0.60087759 0.60111638 0.60195218
 0.60195218 0.60210143]
0.6047337631327603


In [ ]:
from sklearn.ensemble import RandomForestClassifier

param_grid = {'n_estimators': est, 'max_depth': depth}
#bestParams = RF(param_grid, CV_split_X, CV_split_T, fold)


clf = RandomForestClassifier(n_estimators=150, max_depth=None, random_state=0)
clf.fit(X_train, y_train)

y_hat = clf.predict(X_test)
print(accuracy_score(y_test, y_hat))